In [7]:
from pathlib import Path
import nltk as n
import os
import pickle
from datetime import datetime
from nltk.classify import SklearnClassifier
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from sklearn.svm import SVC
startTime = datetime.now()

porter = PorterStemmer()
cwd = str(Path.home())
masterloc=cwd+"/moviereviews/masterfile.txt"
stopwords = n.corpus.stopwords.words('english')
total_set=[]
test_set=[]
dev_set=[]
train_set=[]

    

def machine_features(characters,pos_tagged_s,review,sn,verbsynonym_s):
    #print("in machine feature")
    reviewSentence_list=n.tokenize.sent_tokenize(review)
    reviewlist=n.tokenize.word_tokenize(review)
    tot_features_dic={}
    tot_features_dic.update(char_feature(characters,reviewlist))
    tot_features_dic.update(verb_feature(pos_tagged_s,reviewlist,verbsynonym_s))
    tot_features_dic.update(sentenceVerb_features(characters,pos_tagged_s,reviewSentence_list,verbsynonym_s))
    total_set.append((tot_features_dic,sn))
        
def sentenceVerb_features(characters,pos_tagged_s,reviewSentence_list,verbsynonym_s):

    
    total_sentences = len(reviewSentence_list)
    tot_occ_char_verb = 0
    for i in reviewSentence_list:
        if(len(i)!=0):
            v_bool=False
            c_bool=False
            words = n.tokenize.word_tokenize(i)
            pos_tagged_r = n.pos_tag(words)
            pos_tagged_r=[(w,l) for w,l in pos_tagged_r if w not in stopwords and w.isalnum()]
            pos_tagged_r=[(porter.stem(w),l) for w,l in pos_tagged_r]
            for x in characters:
                x=x.lower()
                if x in words:
                    c_bool=True
                    break        
            for x,y in pos_tagged_r:
                if y in VB_list and x in verbsynonym_s:
                    v_bool=True
                    break
            if c_bool and v_bool:
                tot_occ_char_verb = tot_occ_char_verb + 1
    per_occ_char_verb = tot_occ_char_verb/total_sentences
    return({"percentage_of_occurrence_character_verb":per_occ_char_verb})
    
def char_feature(characters,reviewlist):
    tot_occ_char=0
    per_tot_occ_char=0
    if len(reviewlist)!=0:
        for i in characters:
            i=i.lower()
            tot_occ_char+=reviewlist.count(i)
        per_tot_occ_char=tot_occ_char/len(reviewlist)
    return ({"percentage_of_occurence_character":per_tot_occ_char})

def verb_feature(pos_tagged_s,reviewlist,verbsynonym_s):
    #print("in verb feature")
    pos_tagged_r=n.pos_tag(reviewlist)
    pos_tagged_r=[(w,l) for w,l in pos_tagged_r if (w not in stopwords and w.isalnum())]
    tl=[]
    pos_tagged_r=[(porter.stem(w),l) for w,l in pos_tagged_r]
    tot_occ_verb=0
    per_tot_occ_verb=0
    
    verblist_r=[x for x,y in pos_tagged_r if y in VB_list]
    if len(reviewlist)!=0:
        for i in verblist_r:
            if i in verbsynonym_s:
                tot_occ_verb+=pos_tagged_s.count(i)
        per_tot_occ_verb=tot_occ_verb/len(pos_tagged_r)
    return ({"percentage_of_occurence_verb":per_tot_occ_verb})

    
def trainmachine1(t_set):
    return n.NaiveBayesClassifier.train(train_set)

def trainmachine2(t_set):
    return n.DecisionTreeClassifier.train(train_set)


def testmachine(machine,t_set):
    print("accuracy of the machine is : {}".format(n.classify.accuracy(machine,t_set)))

def storetotalset(t_set):
    total_set_location=cwd+"/moviereviews/total_set.data"
    f=open(total_set_location,"wb")
    pickle.dump(total_set,f)
    f.close()
    

f1 = open(masterloc)
count=0
stopwords
for i in f1.readlines():
    count+=1
    print(count,end=",")
    moviehash=i.rstrip("\n")
    movie_loc=cwd+"/moviereviews/"+moviehash


    movie_n_loc=movie_loc+"/"+moviehash+".txt"
    #movie_n=open(movie_n_loc).readline()


    movie_s_loc=movie_loc+"/"+moviehash+"_s.txt"
    movie_s=open(movie_s_loc).read()

    
    synopsislist=n.word_tokenize(movie_s)                 # pos tagging synopsis early to avoid it doing again and again
    pos_tagged_s=n.pos_tag(synopsislist)
    pos_tagged_s=[(w,l) for w,l in pos_tagged_s if (w not in stopwords and w.isalnum())]
    pos_tagged_s=[(porter.stem(w),l) for w,l in pos_tagged_s]
    VB_list=["VB","VBD","VBG","VBN","VBP","VBZ"]
    verblist_s=[x for x,y in pos_tagged_s if y in VB_list]
    verbsynonym_s = set()
    verbsynonym_s.update(verblist_s)
    for i in verblist_s:
        for j in wordnet.synsets(i):
            for k in j.lemmas():
                verbsynonym_s.add(k.name())
    movie_c_loc=movie_loc+"/"+moviehash+"_c.txt"
    movie_c=[j.rstrip("\n") for j in open(movie_c_loc).readlines()]

    for i in range(25):
        movie_r_loc=movie_loc+"/"+moviehash+"_"+str(i+1)+".txt"
        if os.path.isfile(movie_r_loc)==True:
            f=open(movie_r_loc)
            reviewnumber,sn=f.readline().rstrip("\n").split(",")
            review=f.readline()
            review=review.lower()
            if review!="":

                machine_features(movie_c,pos_tagged_s,review,sn,verbsynonym_s)
            f.close()
f1.close()

storetotalset(total_set)


tt=len(total_set)
#train_set=total_set[:int(0.4*tt)]
#dev_set=total_set[int(0.4*tt)+1:int(0.7*tt)]
#test_set=total_set[int(0.7*tt)+1:]
    
print(" ")
test_set=total_set[:int(0.3*tt)]
train_set=total_set[int(0.3*tt)+1:int(0.7*tt)]
dev_set=total_set[int(0.7*tt)+1:]
        
    
machine1=trainmachine1(train_set)
machine2=trainmachine2(train_set)
testmachine(machine1,test_set)
testmachine(machine1,dev_set)
testmachine(machine2,test_set)
testmachine(machine2,dev_set)



#machine.show_most_informative_features()
print(datetime.now() - startTime)


1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,

In [8]:
test_set=total_set
train_set=total_set
machine1=trainmachine1(train_set)
machine2=trainmachine2(train_set)
machine3=SklearnClassifier(SVC(C=1.0, kernel='linear', degree=3, gamma='auto'), sparse=False).train(train_set)
testmachine(machine1,test_set)
testmachine(machine2,test_set)
testmachine(machine3,test_set)

accuracy of the machine is : 0.7919821471268033
accuracy of the machine is : 0.8327887144337291
accuracy of the machine is : 0.740575436359289


In [9]:
count=0
total=0
detected1=0
notdetected1=0
detected2=0
notdetected2=0
detected3=0
notdetected3=0

counter=0
f1 = open(masterloc)
for i in f1.readlines():
    counter+=1
    print(counter,end=",")
    moviehash=i.rstrip("\n")
    movie_loc=cwd+"/moviereviews/"+moviehash
    
    
    movie_n_loc=movie_loc+"/"+moviehash+".txt"
    movie_n=open(movie_n_loc).readline()
    
    
    movie_s_loc=movie_loc+"/"+moviehash+"_s.txt"
    movie_s=open(movie_s_loc).read()
    synopsislist=n.word_tokenize(movie_s)                 # pos tagging synopsis early to avoid it doing again and again
    pos_tagged_s=n.pos_tag(synopsislist)
    pos_tagged_s=[(w,l) for w,l in pos_tagged_s if w not in stopwords]
    pos_tagged_s=[(porter.stem(w),l) for w,l in pos_tagged_s]
    VB_list=["VB","VBD","VBG","VBN","VBP","VBZ"]
    verblist_s=[x for x,y in pos_tagged_s if y in VB_list]
    verbsynonym_s = set()
    verbsynonym_s.update(verblist_s)
    for i in verblist_s:
        for j in wordnet.synsets(i):
            for k in j.lemmas():
                verbsynonym_s.add(k.name())
    movie_c_loc=movie_loc+"/"+moviehash+"_c.txt"
    movie_c=[j.rstrip("\n") for j in open(movie_c_loc).readlines()]
    
    for i in range(25):
        
        movie_r_loc=movie_loc+"/"+moviehash+"_"+str(i+1)+".txt"
        if os.path.isfile(movie_r_loc)==True:
            f=open(movie_r_loc)
            reviewnumber,sn=f.readline().rstrip("\n").split(",")
            review=f.readline().lower()
            f.close()
            if review!="":
                reviewSentence_list=n.tokenize.sent_tokenize(review)
                reviewlist=n.tokenize.word_tokenize(review)
                tot_features_dic={}
                tot_features_dic.update(char_feature(movie_c,reviewlist))
                tot_features_dic.update(verb_feature(pos_tagged_s,reviewlist,verbsynonym_s))
                tot_features_dic.update(sentenceVerb_features(movie_c,pos_tagged_s,reviewSentence_list,verbsynonym_s))
                t2=machine2.classify(tot_features_dic) 
                t1=machine1.classify(tot_features_dic) 
                #t3=machine3.classify(tot_features_dic) 
                count+=1
                if sn=="S":
                    total+=1
                if t2=="S" and sn=="S":
                    detected2+=1
                if t2=="S" and sn!="S":
                    notdetected2+=1
                if t1=="S" and sn=="S":
                    detected1+=1
                if t1=="S" and sn!="S":
                    notdetected1+=1
                #if t3=="S" and sn=="S":
                #    detected3+=1
                #if t3=="S" and sn!="S":
                #    notdetected3+=1
                
f1.close()
print(" ")
print("detected1",detected1)
print("notdetected1",notdetected1)
print("detected2",detected2)
print("notdetected2",notdetected2)
print("detected3",detected3)
print("notdetected3",notdetected3)
print("total",total)

#print(count)


1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,